In [1]:
from sklearn.datasets import make_classification
import pandas as pd 
import numpy as np 

In [2]:
X, y = make_classification(n_samples=400, n_features=4)
X = pd.DataFrame(X, columns=['f1', 'f2', 'f3', 'f4'])
y = pd.Series(y)

нужно написать методы для предсказаний
* predict
* predict_proba

In [4]:
class MyLogReg():
    def __init__(self, n_iter=10, learning_rate=0.1, weights=None):
        self.n_iter = n_iter 
        self.learning_rate = learning_rate
        self.weights = weights
        
    def __repr__(self):
        return f'MyLogReg class: n_iter={self.n_iter}, learning_rate={self.learning_rate}'
        
    def __log_training_step(self, iteration: int, X: np.array, y: np.array, proba: np.array):
        """Логирует процесс обучения (итерацию, веса, функцию потерь).
    
        Параметры:
        iteration: int - номер итерации
        X: np.array - матрица признаков
        y: np.array - вектор истинных меток (0 или 1)
        proba: np.array - предсказанные вероятности класса 1
        """
        eps = 1e-15
        loss = -np.mean(y*np.log(proba+eps) + (1-y)*(np.log(1-proba+eps)))
        if iteration == 0:
            print(f'start | loss: {loss:0.2f}')
        print(f'{iteration} | loss: {loss:0.2f}')
        
    def fit(self, X: pd.DataFrame, y: pd.Series, verbose=False):
        """
        Метод обучает модель Логистической регресси 
        Входные параметры:
        X: pd.DataFrame
        y: pd.Series
        verbose: bool
        """
        X_copy = X.copy()                                     # копируем матрицу признаков, чтобы не изменить оригинальный 
        X_copy.insert(0, 'base', 1)                           # добавляем столбик для свободного члена, заполним его 1
        X_copy = X_copy.to_numpy()  
        self.weights = np.ones(X_copy.shape[1])               # создаем вектор весов, заполненный 1

        # Цикл обучения
        for i in range(self.n_iter):
            pred = X_copy.dot(self.weights)                    # делаем предсказание модели
            proba = 1 / (1 + np.exp(-pred))                    # переводим предсказания в вероятности через функцию сигмоиды
            grad = (1/len(y))*(proba - y).dot(X_copy)          # вычисляем градиент LogLoss
            self.weights -= self.learning_rate * grad          # делаем шаг обучения

            if verbose and (i % 10 == 0 or i == self.n_iter-1):
                self.__log_training_step(i, X_copy, y, proba)

    def predict_proba(self, X: pd.DataFrame) -> np.array:
        """Вернет вероятностное предсказание модели"""
        X_copy = X.copy()
        X_copy.insert(0, 'base', 1)
        pred = X_copy.dot(self.weights)
        proba = 1 / (1 + np.exp(-pred))
        return np.array(proba)

    def predict(self, X: pd.DataFrame) -> np.array:
        """Вернет маркерное предсказание модели"""
        proba = self.predict_proba(X)
        return np.array((proba > 0.5).astype(int))
                
    def get_coef(self) -> np.array:
        return np.array(self.weights[1:])


In [5]:
model = MyLogReg(n_iter=100)

In [6]:
model

MyLogReg class: n_iter=100, learning_rate=0.1

In [7]:
model.fit(X,y, verbose=True)

start | loss: 1.09
0 | loss: 1.09
10 | loss: 0.62
20 | loss: 0.41
30 | loss: 0.32
40 | loss: 0.27
50 | loss: 0.24
60 | loss: 0.23
70 | loss: 0.21
80 | loss: 0.20
90 | loss: 0.20
99 | loss: 0.19


In [8]:
model.predict_proba(X.iloc[34:40])

array([0.15940069, 0.97368612, 0.19016818, 0.96670751, 0.17597394,
       0.93727563])

In [9]:
model.predict(X.iloc[34:40])

array([0, 1, 0, 1, 0, 1])

In [10]:
X.iloc[2:5]

,f1,f2,f3,f4
2,-1.582038,-0.386774,1.792626,0.315114
3,-0.975066,-0.216205,1.322716,0.737870
4,-2.317061,-0.681481,1.495707,-2.357795


In [11]:
y.iloc[34:40]

34    0
35    1
36    0
37    1
38    0
39    1
dtype: int32